In [52]:
# import packages 
import pandas as pd

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV


import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv("titanic.csv")

df=df.drop('PassengerId',axis=1).drop('Name',axis=1).drop('Ticket',axis=1).drop('Cabin',axis=1).drop('Embarked',axis=1).dropna()
df['Sex']=df['Sex'].replace('male',0).replace('female',1)
df


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,0,22.0,1,0,7.2500
1,1,1,1,38.0,1,0,71.2833
2,1,3,1,26.0,0,0,7.9250
3,1,1,1,35.0,1,0,53.1000
4,0,3,0,35.0,0,0,8.0500
...,...,...,...,...,...,...,...
885,0,3,1,39.0,0,5,29.1250
886,0,2,0,27.0,0,0,13.0000
887,1,1,1,19.0,0,0,30.0000
889,1,1,0,26.0,0,0,30.0000


In [53]:
X = df.drop("Survived", axis=1).values
y = df["Survived"].values

r = 4
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)



In [54]:
# bagged tree
base = DecisionTreeClassifier(max_depth=5)
ensemble = BaggingClassifier(base_estimator=base, n_estimators=100, random_state=7)

base.fit(X_train,y_train)
ensemble.fit(X_train,y_train)

print("Accuracy base:",base.score(X_test, y_test))
print("Accuracy ensemble:",ensemble.score(X_test, y_test))

Accuracy base: 0.7972027972027972
Accuracy ensemble: 0.8321678321678322


In [55]:
#boosted tree
boosted_tree = AdaBoostClassifier(n_estimators=100)
boosted_tree.fit(X_train, y_train)
print("Accuracy boosted tree:",boosted_tree.score(X_test, y_test))


Accuracy boosted tree: 0.8251748251748252


In [1]:
#random forest
forest = RandomForestClassifier(n_estimators=100, random_state=7)
forest.fit(X_train, y_train)
print("Accuracy random forest tree:",forest.score(X_test, y_test))

feature_imp = pd.Series(forest.feature_importances_).sort_values(ascending=False)
feature_imp

NameError: name 'RandomForestClassifier' is not defined

The second feature contributes the most to predicting whether a passenger survives or not

In [57]:
#Determination of optimal parameters
param_grid = {'n_estimators': [75, 100, 125, 150, 200],
              'max_depth': [4, 6, 8, 10]}

forest = RandomForestClassifier()

grid_search = GridSearchCV(forest, param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print('Best n_ estimators parameter',best_params['n_estimators'])
print('Best max depth parameter',best_params['max_depth'])

rf_best = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                                  max_depth=best_params['max_depth'])
rf_best.fit(X_train, y_train)

print("Accuracy Random forest with optimal parameters: ",rf_best.ascore(X_test, y_test))

Best n_ estimators parameter 75
Best max depth parameter 10
Accuracy Random forest with optimal parameters:  0.8181818181818182


The random forest model with the optimised parameters performed the best